In [26]:
# ======================================================
# 백테스팅 (DOGEUSDT)
# ======================================================

import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path
import traceback
import warnings

# 경고 메시지 간소화 (선택사항)
warnings.filterwarnings('ignore', category=UserWarning)

# ------------------------------------------------------
# 1. 환경 설정 및 임포트
# ------------------------------------------------------
# 현재 작업 경로를 sys.path에 추가하여 로컬 모듈 인식
sys.path.append(os.getcwd())

# 시각화 스타일 설정
plt.style.use('seaborn-v0_8-darkgrid')

# Quant 폴더 내 모듈 임포트
# (아래 파일들이 현재 노트북과 같은 폴더 또는 Python 경로 내에 있어야 함)
from data_loader import DataLoader, DataPaths
from feature_store import FeatureStore
from strategy_base import Strategy
from execution import ExecutionEngine
from backtest_engine import BacktestEngine
from metrics import compute_metrics, plot_cum_pnl

# ------------------------------------------------------
# 2. 사용자 파라미터 설정 (Configuration)
# ------------------------------------------------------
DATA_ROOT = r"C:\Users\elvin\OneDrive - SKKU\Quant\백테스팅 데이터\um_daily"

SYMBOL = "DOGEUSDT"
START_DATE = "2024-01-01"
END_DATE   = "2024-02-01"

INITIAL_CAPITAL = 100.0  # 초기 자본금 ($)

# ------------------------------------------------------
# 3. CustomDataLoader 정의 (핵심 수정 사항 포함)
# ------------------------------------------------------
class CustomDataLoader(DataLoader):
    def __init__(self, data_root):
        super().__init__(data_root)
        # 사용자 폴더 구조에 맞춰 경로 객체 재정의
        self.paths = DataPaths(
            root=Path(data_root),
            agg_trades_dir="aggTrades",
            book_ticker_dir="bookTicker",
            klines_1m_dir="klines"
        )

    # [수정 1] BookTicker 로드: 중복 컬럼 처리 & 시간 파싱 에러 해결
    def load_book_ticker_day(self, symbol: str, ymd: str) -> pd.DataFrame:
        fp = self.paths.ticker_day(symbol, ymd)
        if not fp.exists(): 
            return pd.DataFrame()
            
        try:
            df = pd.read_parquet(fp).copy()

            # (A) 컬럼 이름 매핑 (Duplicate keys 방지)
            rename_map = {
                "best_bid_price": "bid_p", "best_bid_qty": "bid_q",
                "best_ask_price": "ask_p", "best_ask_qty": "ask_q"
            }
            # transaction_time을 우선 사용, event_time은 충돌 방지를 위해 제거
            if "transaction_time" in df.columns:
                rename_map["transaction_time"] = "ts"
                if "event_time" in df.columns: 
                    df = df.drop(columns=["event_time"])
            elif "event_time" in df.columns:
                rename_map["event_time"] = "ts"

            df = df.rename(columns=rename_map)

            # (B) 시간 컬럼 파싱 (문자열 Unix Timestamp 문제 해결)
            if "ts" in df.columns:
                # 1. 일단 숫자로 강제 변환 (Coerce errors -> NaT/NaN)
                #    문자열 "1704..." 형태도 여기서 숫자로 바뀝니다.
                ts_numeric = pd.to_numeric(df["ts"], errors='coerce')
                
                # 2. 변환된 값들이 유효한 숫자인지 확인
                if ts_numeric.notna().sum() > 0:
                    # 숫자라면 ms 단위 Timestamp로 변환
                    df["ts"] = pd.to_datetime(ts_numeric, unit="ms", utc=True)
                else:
                    # 숫자가 아니라면 일반 날짜 문자열로 간주하고 파싱
                    df["ts"] = pd.to_datetime(df["ts"], utc=True, errors="coerce")

            # (C) 가격/수량 수치형 변환
            for c in ["bid_p", "bid_q", "ask_p", "ask_q"]:
                if c in df.columns:
                    df[c] = pd.to_numeric(df[c], errors="coerce").astype("float64")

            df["symbol"] = symbol
            # 타임스탬프 기준 정렬
            if "ts" in df.columns:
                df = df.sort_values("ts").reset_index(drop=True)
            
            return df
            
        except Exception as e:
            print(f"[ERROR] Failed to load bookTicker: {fp} ({e})")
            return pd.DataFrame()

    # [수정 2] 1분봉 로드: 파일명 규칙(_) 및 KST 시간대 처리
    def load_klines_1m_day(self, symbol: str, ymd: str) -> pd.DataFrame:
        # 파일명 형식: DOGEUSDT_2024_01_01.parquet
        ymd_underscore = ymd.replace("-", "_")
        file_path = self.paths.root / self.paths.klines_1m_dir / symbol / "1m" / f"{symbol}_{ymd_underscore}.parquet"
        
        if not file_path.exists():
            return pd.DataFrame()
            
        try:
            df = pd.read_parquet(file_path).copy()
            
            # 시간 컬럼 처리 (open_time_kst 우선)
            if "open_time_kst" in df.columns:
                # 이미 datetime 객체인지 확인
                if pd.api.types.is_datetime64_any_dtype(df["open_time_kst"]):
                    df["open_ts"] = df["open_time_kst"].dt.tz_convert("UTC")
                else:
                    # 문자열/객체라면 변환
                    df["open_ts"] = pd.to_datetime(df["open_time_kst"], utc=True)
            elif "open_time" in df.columns:
                # Unix Timestamp(ms) vs 문자열 자동 감지
                df["open_ts"] = pd.to_datetime(df["open_time"], unit="ms", utc=True, errors='coerce')
                if df["open_ts"].isna().all(): # 만약 unit=ms로 안되면 일반 파싱 재시도
                     df["open_ts"] = pd.to_datetime(df["open_time"], utc=True, errors='coerce')
            elif "open_time_ms" in df.columns:
                df["open_ts"] = pd.to_datetime(df["open_time_ms"], unit="ms", utc=True)
            
            # OHLCV 수치형 변환
            for col in ["open", "high", "low", "close", "volume"]:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors="coerce").astype("float64")

            df["symbol"] = symbol
            if "open_ts" in df.columns:
                df = df.sort_values("open_ts").reset_index(drop=True)
            return df
        except Exception as e:
            print(f"[ERROR] Failed to load klines: {file_path} ({e})")
            return pd.DataFrame()

# ------------------------------------------------------
# 4. 엔진 초기화 (Engine Setup)
# ------------------------------------------------------
print(">>> Initializing Backtest Components...")

# 커스텀 로더 인스턴스 생성
loader = CustomDataLoader(DATA_ROOT)

# Feature Store 설정
feature_store = FeatureStore(
    symbol=SYMBOL,
    signal_interval_seconds=5 * 60,  # 5분 단위 시그널 계산
    vpin_bucket_volume=1e6,          # VPIN 버킷 크기
    vpin_history=100,
    tc_window=60,
    ofi_window_minutes=5,
    ofi_z_window=30,
)

# 실행(체결) 엔진 설정
execution = ExecutionEngine(
    latency_ms=20,       # 체결 지연 시간 (ms)
    slippage_bp=1.0,     # 슬리피지 (bp)
)

# 전략 설정
strategy = Strategy(
    symbol=SYMBOL,
    feature_store=feature_store,
    initial_capital=INITIAL_CAPITAL,
    signal_interval_seconds=5 * 60,
    max_holding_seconds=240,
    target_bp=10.0,
)

# 백테스트 메인 엔진
engine = BacktestEngine(
    loader=loader,
    strategy=strategy,
    execution=execution,
    feature_store=feature_store,
    initial_capital=INITIAL_CAPITAL,
    verbose=True
)

# ------------------------------------------------------
# 5. 백테스트 실행 (Execution Loop)
# ------------------------------------------------------
dates = pd.date_range(start=START_DATE, end=END_DATE, freq="D").strftime("%Y-%m-%d").tolist()
print(f"\n[INFO] Backtest Period: {START_DATE} ~ {END_DATE} ({len(dates)} days)")

all_fills = []
all_closed_trades = []

for day in dates:
    print(f"\n>> RUNNING: {SYMBOL} {day}")
    try:
        # 하루치 데이터 로드 및 시뮬레이션
        engine.run_day(symbol=SYMBOL, ymd=day)
        
        # 결과 수집
        if engine.fills:
            all_fills.extend(engine.fills)
        if engine.closed_trades:
            all_closed_trades.extend(engine.closed_trades)
            
        # (옵션) 메모리 관리를 위해 엔진 내부 리스트 초기화 가능
        # engine.fills = []
        # engine.closed_trades = []

    except FileNotFoundError as e:
        print(f"   [SKIP] Data missing for {day}: {e}")
    except Exception as e:
        print(f"   [ERROR] Failed to run {day}")
        traceback.print_exc()

# ------------------------------------------------------
# 6. 결과 분석 및 시각화 (Analysis)
# ------------------------------------------------------
print("\n" + "="*50)
print("             BACKTEST RESULTS")
print("="*50)

fills_df = pd.DataFrame(all_fills)
closed_df = pd.DataFrame(all_closed_trades)

print(f"Total Fills: {len(fills_df)}")
print(f"Total Closed Trades: {len(closed_df)}")

if not closed_df.empty:
    # 종료 시간 기준 정렬
    closed_df_sorted = closed_df.sort_values("exit_ts")
    
    # 누적 수익(PnL) 및 자산 곡선(Equity Curve) 계산
    closed_df_sorted['cumulative_pnl'] = closed_df_sorted['net_pnl'].cumsum()
    closed_df_sorted['equity'] = INITIAL_CAPITAL + closed_df_sorted['cumulative_pnl']
    
    # Metrics 모듈을 위한 데이터프레임 (ts, equity)
    equity_curve = closed_df_sorted[['exit_ts', 'equity']].rename(columns={'exit_ts': 'ts'})
    
    # 성과 지표 산출
    metrics_result = compute_metrics(equity_curve, initial_capital=INITIAL_CAPITAL)
    
    print("\n[Performance Metrics]")
    for k, v in metrics_result.items():
        print(f" - {k:<20}: {v}")
    
    # 수익 곡선 그래프
    print("\n[Plotting Equity Curve]")
    plot_cum_pnl(
        equity_curve,
        initial_capital=INITIAL_CAPITAL,
        title=f"{SYMBOL} Strategy Result ({START_DATE} ~ {END_DATE})"
    )
    
    # 최근 거래 내역 확인
    print("\n[Last 5 Closed Trades]")
    display(closed_df_sorted.tail())

else:
    print("\n[INFO] No trades were closed during the simulation.")
    if not fills_df.empty:
        print("However, there are open positions or unfilled orders. Checking 'fills_df':")
        display(fills_df.head())

>>> Initializing Backtest Components...

[INFO] Backtest Period: 2024-01-01 ~ 2024-02-01 (32 days)

>> RUNNING: DOGEUSDT 2024-01-01
[ORDER] 2024-01-01 00:05:00.190000+00:00 DOGEUSDT MARKET id=1 (compute+10ms)
[FILL ENTER] 2024-01-01 00:05:02.074000+00:00 DOGEUSDT price=0.0898 fee_bp=5.00
[ORDER] 2024-01-01 00:09:04.306000+00:00 DOGEUSDT MARKET id=2 (compute+10ms)
[FILL EXIT ] 2024-01-01 00:09:08.554000+00:00 DOGEUSDT gross=0.0468 net=-0.0532 fee_bp=5.00
[ORDER] 2024-01-01 00:10:00.563000+00:00 DOGEUSDT MARKET id=3 (compute+10ms)
[FILL ENTER] 2024-01-01 00:10:03.455000+00:00 DOGEUSDT price=0.0899 fee_bp=5.00
[ORDER] 2024-01-01 00:14:05.931000+00:00 DOGEUSDT MARKET id=4 (compute+10ms)
[FILL EXIT ] 2024-01-01 00:14:06.310000+00:00 DOGEUSDT gross=0.0690 net=-0.0310 fee_bp=5.00
[ORDER] 2024-01-01 00:15:01.343000+00:00 DOGEUSDT MARKET id=5 (compute+10ms)
[FILL ENTER] 2024-01-01 00:15:02.235000+00:00 DOGEUSDT price=0.0898 fee_bp=5.00
[ORDER] 2024-01-01 00:19:02.429000+00:00 DOGEUSDT MARKET id

KeyboardInterrupt: 